In [1]:
import sys
!{sys.executable} -m pip install pulp 

import pulp
# Import PuLP modeler functions
from pulp import * 

import numpy
import matplotlib as plt
import pandas as pd

# Seaborn for pretty graphs
import seaborn

# May do plotly instead for interactivity

## Context: Marine Spatial Planning and a Linear Programming Approach

## The Decision Variables

In [2]:
"""
Strait of Georgia, Northern Shelf, Southern Shelf and Offshore Coast are the 4 main bioregions localized to the 
BC region.
"""

bioregions = ['SG', 'NS', 'SS', 'OC']

In [3]:
prob = LpProblem("Marine_Spatial_Planning_BC",LpMinimize)

In [4]:
#lowBound = 0 to enforce nonnegativity
msp_areas = LpVariable.dicts("bioregions",bioregions,lowBound=0)

## The Objective Function

In [5]:
prob+=lpSum([msp_areas[i] for i in bioregions]),"Minimum Area Required"

## The Constraints

In [6]:
# PROJECTED NUMBERS

#Strait of Georgia = 6,800 km2
#Northern Shelf = 101,000 km2
#Southern Shelf = 41, 562 km2
#Offshore Coast = 335,825 km2

bioregion_area = {'SG': 6800,
                  'NS': 101000,
                  'SS': 41562,
                  'OC': 335825}

for j in msp_areas:
    prob += msp_areas[j] <= bioregion_area[j]



#75% of combined land in strait of Georgia, Northern and Southern shelf should accomodate seabirds
seabird_areas = { 'SG': 1,
                  'NS': 1,
                  'SS': 1,
                  'OC': 0}

prob += lpSum([seabird_areas[i] * msp_areas[i] for i in msp_areas]) >= 112021.5



#45% of combined land in Northern and Southern shelf should accomodate marine fish life and plantlife
shelf_fish_areas = {'SG': 0,
                    'NS': 1,
                    'SS': 1,
                    'OC': 0}

prob += lpSum([shelf_fish_areas[i] * msp_areas[i] for i in msp_areas]) >= 64152.9



#20% of combined land in Georgia and Offshore coast should accomodate marine fish life and plantlife
other_fish_areas = {'SG': 1,
                    'NS': 0,
                    'SS': 0,
                    'OC': 1}

prob += lpSum([other_fish_areas[i] * msp_areas[i] for i in msp_areas]) >= 68525.0

#21.9% of combined land in Georgia and Offshore coast should accomodate sector use


In [7]:
print(prob)

#LP also written to .lp file for convenience
prob.writeLP("BC_MSP.lp")

Marine_Spatial_Planning_BC:
MINIMIZE
1*bioregions_NS + 1*bioregions_OC + 1*bioregions_SG + 1*bioregions_SS + 0
SUBJECT TO
_C1: bioregions_SG <= 6800

_C2: bioregions_NS <= 101000

_C3: bioregions_SS <= 41562

_C4: bioregions_OC <= 335825

_C5: bioregions_NS + bioregions_SG + bioregions_SS >= 112021.5

_C6: bioregions_NS + bioregions_SS >= 64152.9

_C7: bioregions_OC + bioregions_SG >= 68525

VARIABLES
bioregions_NS Continuous
bioregions_OC Continuous
bioregions_SG Continuous
bioregions_SS Continuous



[bioregions_NS, bioregions_OC, bioregions_SG, bioregions_SS]

## Solution to the LP

In [8]:
prob.solve()
print("Status:",LpStatus[prob.status])

for a in prob.variables():
    print(a.name,"=",a.varValue)
    
print("Optimal area amount = ",value(prob.objective))

Status: Optimal
bioregions_NS = 101000.0
bioregions_OC = 61725.0
bioregions_SG = 6800.0
bioregions_SS = 4221.5
Optimal area amount =  173746.5


## Accounting for minute changes in our variables - Sensitivity Analysis